In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

from scipy.spatial.distance import pdist, squareform
from scipy.spatial import distance_matrix
from itertools import permutations, combinations
from math import comb
import numpy as np
from timeit import default_timer as timer
import sys
sys.path.insert(0, '/home/kurk/curse/filtration/graph_func.cpython-312-x86_64-linux-gnu.so')

import netlibpp_cpy as gf
import importlib
importlib.reload(gf)
from timeit import default_timer as timer

from sklearn.neighbors import radius_neighbors_graph
import networkx as nx


print(gf.__file__)
print(gf.filtrate.__doc__)

In [ ]:
X = np.array([
    [0, 2.75], # 1
    [2, 2], # 2
    [3, 2.25], # 3
    [4, 4], #4
    [3.5, 0.75], #5
    [2.5, -0.25], #6
    [2.25, -2.25], #7
    [2.5, -4], #8
    [1, -3], #9
    [-1, -2.75], #10
    [-2.5, -1.0], #11
    [-4, 0.5], #12
    [-2, 1] #13
])

X = X + np.random.normal(0, 0.05, X.shape)
X

In [ ]:
A = radius_neighbors_graph(X, 4.25, mode='distance', include_self=False).toarray() #.astype(int)
G = nx.from_numpy_array(A)
A

In [ ]:
_A = distance_matrix(X, X)
dm = gf.get_VR_from_dist_matrix(_A, 0, 3)
# dm.creat(A, 3)
# dm.as_list()
_A

In [ ]:
def plot_core(G, K):

    def stack(idx):
        ret = np.empty((0, 2))
        for _id in idx:
            ret = np.vstack((ret, X[_id,:]))
        return ret

    max_dim_K = len(K)

    plt.figure(figsize=(17,4))

    plt.subplot(1, 4, 1)
    plt.xlim((-5, 5))
    plt.ylim((-5, 5))
    plt.grid(linestyle="dotted")
    plt.title("Neighborhood graph$")

    # plot vertices
    plt.scatter(X[:,0], X[:,1])

    for i in range(len(X)):
        plt.annotate(str(i), (X[i][0], X[i][1]))

    # plot edges
    for e in G.edges():
        (start_id, end_id) = e
        plt.plot([X[start_id,0], X[end_id,0]], [X[start_id,1], X[end_id,1]], 'c-', alpha=0.5)

    plt.subplot(1, 4, 3)
    plt.xlim((-5, 5))
    plt.ylim((-5, 5))
    plt.grid(linestyle="dotted")
    plt.title("Vietoris-Rips complex$")

    # plot vertices
    plt.scatter(X[:,0], X[:,1])

    # plot edges
    for e in G.edges():
        (start_id, end_id) = e
        plt.plot([X[start_id,0], X[end_id,0]], [X[start_id,1], X[end_id,1]], 'c-', alpha=0.5)

    # plot triangles
    if max_dim_K >= 3:
        for triangle in K[2]:
            t = plt.Polygon(stack(triangle), color="blue", alpha=0.15)
            plt.gca().add_patch(t)

    # plot tetrahedrons
    if max_dim_K >= 4:
        for tetra in K[3]:
            tt = plt.Polygon(stack(tetra), color="red", alpha=0.15)
            plt.gca().add_patch(tt)

    plt.show()

In [ ]:
epsilons = list(np.sort(A[np.tril(A).nonzero()]))
epsilons

In [ ]:
_A = radius_neighbors_graph(X, epsilons[len(epsilons) - 1], mode='distance', include_self=False).toarray()
__A = distance_matrix(X, X)
__K = gf.get_VR_from_dist_matrix(__A, epsilons[len(epsilons) - 1], 4)
_K =  __K.as_index_list()
plot_core(nx.from_numpy_array(_A), _K)
print(__K.filtration(2)[5].dim())
__K.filtration(1)[5].get_volume()

In [ ]:
__K = gf.get_Lp_from_coord_matrix(X, 4.23, 2, 4)
__K.as_index_list()

In [ ]:
point = gf.Point(np.array([0, 0]).astype(np.float64))
pr = __K.projection(point)
for i in pr:
    print(i.coords())

In [ ]:
l = __K.as_list()
__K.simplex_from_indexes(l[2][0]).get_volume()
print(__K.boundary_matrix(1).shape, __K.boundary_matrix(2).shape)
__K.laplace_matrix(1)

In [ ]:
__K = 0

ttt = np.array([[0, 0], [2, 0], [1, 3 ** 0.5], [1, 3 ** 0.5 + 1]])

min_dist = 2.85

__K = gf.get_Lp_from_coord_matrix(ttt, min_dist, 2, 4)
__V = gf.get_VR_from_coord_matrix(ttt, min_dist, 4)
print(__K.as_index_list())
print(__V.as_index_list())

In [ ]:
__K = 0

for epsilon in epsilons:
    _A = radius_neighbors_graph(X, epsilon, mode='distance', include_self=False).toarray() #.astype(int)

    _G = nx.from_numpy_array(_A)

    _M = _A.copy()
    np.fill_diagonal(_M, 1)

    print(_G)
    _A = _A.astype(np.float32)
    __K = gf.get_VR_from_coord_matrix(X, epsilon, 4)
    _K =  __K.as_index_list()
    # plot_core(_G, _K)

In [ ]:
__K = 0

for epsilon in epsilons:
    _A = radius_neighbors_graph(X, epsilon, mode='distance', include_self=False).toarray() #.astype(int)

    _G = nx.from_numpy_array(_A)

    print(_G)
    __K = gf.get_Lp_from_coord_matrix(X, epsilon, 2, 4)
    _K =  __K.as_index_list()
    # plot_core(_G, _K)